In [1]:
path = '/home/kanit/mimic3/DB_22000_2'
block = 0

In [2]:
from utils import *
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

# connect DB

In [3]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


# load admissionID 

In [4]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

# load filtered itemID

In [5]:
v = np.load('res/filtered_input.npy').tolist()
valid_input = v['id']
valid_input_unit = v['unit']

v = np.load('res/filtered_output.npy').tolist()
valid_output = v['id']

v = np.load('res/filtered_chart.npy').tolist()
valid_chart = v['id']
valid_chart_unit = v['unit']

v = np.load('res/filtered_chart_cate.npy').tolist()
valid_chart_cate = v['id']

v = np.load('res/filtered_lab.npy').tolist()
valid_lab = v['id']
valid_lab_unit = v['unit']

v = np.load('res/filtered_lab_num.npy').tolist()
valid_lab_num = v['id']
valid_lab_num_unit = v['unit']

v = np.load('res/filtered_lab_cate.npy').tolist()
valid_lab_cate = v['id']

v = np.load('res/filtered_microbio.npy').tolist()
valid_microbio = v['id']

v = np.load('res/filtered_prescript.npy').tolist()
valid_prescript = v['id']
valid_prescript_unit = v['unit']
allids = valid_input+valid_output+valid_chart+valid_chart_cate+valid_lab+valid_lab_num+valid_lab_cate+valid_microbio+valid_prescript

# list final number of parameters

In [6]:
print 'len(valid_input) = '+ str(len(valid_input))
print 'len(valid_output) = '+ str(len(valid_output))
print 'len(valid_chart) = '+ str(len(valid_chart))
print 'len(valid_chart_cate) = '+ str(len(valid_chart_cate))
print 'len(valid_lab) = '+ str(len(valid_lab))
print 'len(valid_lab_num) = '+ str(len(valid_lab_num))
print 'len(valid_lab_cate) = '+ str(len(valid_lab_cate))
print 'len(valid_microbio) = '+ str(len(valid_microbio))
print 'len(valid_prescript) = '+ str(len(valid_prescript))
print '\nlen(allids) = '+ str(len(allids) )

len(valid_input) = 277
len(valid_output) = 68
len(valid_chart) = 660
len(valid_chart_cate) = 1
len(valid_lab) = 372
len(valid_lab_num) = 54
len(valid_lab_cate) = 231
len(valid_microbio) = 9085
len(valid_prescript) = 2371

len(allids) = 13119


In [7]:
admission_ids_txt =""
for t in admission_ids:
    admission_ids_txt += str(t)+","
admission_ids_txt=admission_ids_txt[0:-1]

valid_input_txt =""
for t in valid_input:
    valid_input_txt += str(t)+","
valid_input_txt=valid_input_txt[0:-1]

valid_output_txt =""
for t in valid_output:
    valid_output_txt += str(t)+","
valid_output_txt=valid_output_txt[0:-1]

valid_chart_txt =""
for t in valid_chart:
    valid_chart_txt += str(t)+","
valid_chart_txt=valid_chart_txt[0:-1]

valid_chart_cate_txt =""
for t in valid_chart_cate:
    valid_chart_cate_txt += str(t)+","
valid_chart_cate_txt=valid_chart_cate_txt[0:-1]

valid_lab_txt =""
for t in valid_lab:
    valid_lab_txt += str(t)+","
valid_lab_txt=valid_lab_txt[0:-1]

valid_lab_num_txt =""
for t in valid_lab_num:
    valid_lab_num_txt += str(t)+","
valid_lab_num_txt=valid_lab_num_txt[0:-1]

valid_lab_cate_txt =""
for t in valid_lab_cate:
    valid_lab_cate_txt += str(t)+","
valid_lab_cate_txt=valid_lab_cate_txt[0:-1]

valid_microbio_txt ="'"
for t in valid_microbio:
    valid_microbio_txt += str(t)+"','"
valid_microbio_txt=valid_microbio_txt[0:-2]

valid_prescript_txt ="'"
for t in valid_prescript:
    valid_prescript_txt += str(t)+"','"
valid_prescript_txt=valid_prescript_txt[0:-2]

In [8]:
index =0
map_itemid_index = {}
allitem = valid_output + valid_input + valid_chart + valid_chart_cate + valid_lab + valid_lab_num + valid_lab_cate + valid_microbio + valid_prescript
allitem_unit = ['NOCHECK']*len(valid_output) + valid_input_unit + valid_chart_unit +['NOCHECK'] + valid_lab_unit + valid_lab_num_unit+ ['NOCHECK']*len(valid_lab_cate)+ ['NOCHECK']*len(valid_microbio) + valid_prescript_unit
for i in range(len(allitem_unit)):
    allitem_unit[i] = allitem_unit[i].replace(' ','').lower()
assert len(allitem) == len(allitem_unit)
for ai in allitem:
    map_itemid_index[ai] = index
    index+=1

For the categorical feature, Here we give a number for each different value. For example, A, B, C, D, we assign A to 1, B to 2, C to 3, D to 4.

In [9]:
catedict = {}

for i in valid_chart_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+')')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print '.',
    
for i in valid_lab_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+')')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print '.',
print "DONE"

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . DONE


# This method is the main part of this file.
input : 
- aid = admission ID that we would like to process,
- f = file to print out log. (log can be very long)

output: 
- matrix represent the all events in admission.

### There are 2 stage of processing.
#### 1. gathering all events

In this step, we have an array called whole data, it keep all the events from every tables. For each table we query the observations for the admission aid, we place it into this format.

[type of event, time after admission (in seconds), list(time,itemid,values,uom)]

for example : ['oe',1,(01/01/2010 1pm, 1234, 15,'mL')]
It means the records is from outputevent table, 1 second after admission time, the event is at 1/1/2010 1pm, itemid=1234, output is 15 mL.

once we have the array. we can know all the distinct timestep, then we can know the size of the matrix that we have to prepare.

we build the mapping on time to row number, so that given a time we can exactly know where to put the value in.

**In this step, we also preprocess some number.** In this part, we make sure that every number passed to step 2 are actually numerical. For example, if the input value is '0-2', we will average it so the value will be equal to 1. To parse the number, we write our own function, which is in utils.py.

**Also if the unit of measurement are not match with the main uom, then we try to convert it**. If it is not convertable, then we discard that observation. The conversion table is in utils.py file.

#### 2. place to the array

Basically, just place everything on the call corresponding to the time and itemid of that observations.

We also need to beware that if there are multiple observation of the same itemid at the same time. We need to take some action (sum the value or average the value). It depends on the variables, if it is from chartevent, we will average them. If it is from outputevents, we will sum them up.

Finally, We got the array.


In [ ]:
def processing(aid,f=sys.stdout):
    
    # init array to keep dropped observation
    dropobservation = [0] * len(allitem)
    
    # admission
    cur = conn.cursor()
    cur.execute('SELECT * FROM mimiciii.admissions WHERE hadm_id='+str(aid))
    admission = cur.fetchone()
    admitdate = admission[3]
    if(admitdate == None):
        return None;
    writeline(f,"aid : "+ str(aid))
    wholedata = []

    #preprocess inputevents
    cur = conn.cursor()
    cur.execute('SELECT starttime,itemid,amount,amountuom FROM mimiciii.inputevents_mv WHERE amount>0 and hadm_id = '+str(aid)+' and itemid in ('+valid_input_txt+')')
    inputevents = cur.fetchall()
    for ie in inputevents : 
        
        #check date
        if(ie[0] == None):
            writeline(f,'ie date 0 : ' + " : "+str(ie))
            dropobservation[map_itemid_index[ie[1]]] +=1
            continue;
        
        #discard none value
        if(ie[2] == None):
            writeline(f,'INPUT None value :' + str(ie))
            dropobservation[map_itemid_index[ie[1]]] +=1
            continue;

        #tuple to list
        ie = list(ie)
        
        #check unit
        currentunit = ie[3].replace(' ','').lower()
        if( currentunit == allitem_unit[map_itemid_index[ie[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[ie[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[ie[1]]];
                ie[2] = ie[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'INPUT Unit not match : ' + ie[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[ie[1]]] + " : "+str(ie))
                dropobservation[map_itemid_index[ie[1]]] +=1
                continue;

        #discard none value
        if(ie[2] == None):
            writeline(f,'INPUT None value :' + str(ie))
            dropobservation[map_itemid_index[ie[1]]] +=1
            continue;
            
        wholedata.append(['ie',(ie[0]-admitdate).total_seconds(),list(ie)])

    #preprocess outputevents
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value FROM mimiciii.outputevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_output_txt+')')
    outputevents = cur.fetchall()
    for oe in outputevents : 
        #check date
        if(oe[0] == None):
            writeline(f,'oe date 0 : ' + " : "+str(oe))
            dropobservation[map_itemid_index[oe[1]]] +=1
            continue;
        
        #discard none value
        if(oe[2] == None):
            writeline(f,'OUTPUT None value :' + str(oe))
            dropobservation[map_itemid_index[oe[1]]] +=1
            continue;
            
        # no need to check unit all is mL
        wholedata.append(['oe',(oe[0]-admitdate).total_seconds(),list(oe)])

    #preprocess chartevents
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_chart_txt+')')
    chartevents = cur.fetchall()
    for ce in chartevents: 
        
        #check date
        if(ce[0] == None):
            writeline(f,'ce date 0 : ' + " : "+str(ce))
            dropobservation[map_itemid_index[ce[1]]] +=1
            continue;
        
        #discard none value
        if(ce[2] == None):
            writeline(f,'CHART None value :' + str(ce))
            dropobservation[map_itemid_index[ce[1]]] +=1
            continue;
        
        #tuple to list
        ce = list(ce)
        
        #check unit
        currentunit = ce[3].replace(' ','').lower()
        if( currentunit == allitem_unit[map_itemid_index[ce[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[ce[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[ce[1]]];
                ce[2] = ce[2] / gr[currentunit] * gr[targetunit]
                writeline(f,'******************************************' + str(ce))
            else:
                writeline(f,'CHART Unit not match : ' + ce[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[ce[1]]] + ' : ' + " : "+str(ce))
                dropobservation[map_itemid_index[ce[1]]] +=1
                continue;
        
        #discard none value
        if(ce[2] == None):
            writeline(f,'CHART None value :' + str(ce))
            dropobservation[map_itemid_index[ce[1]]] +=1
            continue;
        
        wholedata.append(['ce',(ce[0]-admitdate).total_seconds(),list(ce)])

        
    #preprocess chartevents cate
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_chart_cate_txt+')')
    chartevents = cur.fetchall()
    for ce in chartevents: 
        
        #check date
        if(ce[0] == None):
            writeline(f,'cecate date 0 : ' + " : "+str(ce))
            dropobservation[map_itemid_index[ce[1]]] +=1
            continue;
        
        #discard none value
        if(ce[2] == None):
            writeline(f,'cecate None value :' + str(ce))
            dropobservation[map_itemid_index[ce[1]]] +=1
            continue;
        
        #tuple to list
        ce = list(ce)
        
        #map to num
        ce[2] = catedict[ce[1]][ce[2]]
        wholedata.append(['cecate',(ce[0]-admitdate).total_seconds(),list(ce)])

            
    #preprocess labevents
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_lab_txt+')')
    labevents = cur.fetchall()
    for le in labevents :    
        #check date
        if(le[0] == None):
            writeline(f,'le date 0 : ' + " : "+str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;

        #discard none value
        if(le[2] == None):
            writeline(f,'LAB None value :' + str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;
    
        #tuple to list
        le = list(le)
        
        #check unit
        currentunit = le[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        if( currentunit == allitem_unit[map_itemid_index[le[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            
            # try to convert if unit is different and convertible.
            if(allitem_unit[map_itemid_index[le[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[le[1]]];
                le[2] = le[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'LAB Unit not match : ' + le[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[le[1]]] + " : " + " : "+str(le))
                dropobservation[map_itemid_index[le[1]]] +=1
                continue;
        
        #discard none value
        if(le[2] == None):
            writeline(f,'LAB None value :' + str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;
        
        #discard none value
        if(le[2] == None):
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;
        
        wholedata.append(['le',(le[0]-admitdate).total_seconds(),list(le)])
        
    #preprocess labevents_num
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_lab_num_txt+')')
    labevents = cur.fetchall()
    for le in labevents :    
        
        #tuple to list
        le = list(le)
        
        #translate values
        le[2] = parseNum(le[2]);
        
        #check date
        if(le[0] == None):
            writeline(f,'lenum date 0 : ' + " : "+str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;

        #discard none value
        if(le[2] == None):
            writeline(f,'lenum None value :' + str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;
    
        #check unit
        currentunit = le[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        if( currentunit == allitem_unit[map_itemid_index[le[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            # try to convert if unit is different and convertible.
            if(allitem_unit[map_itemid_index[le[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[le[1]]];
                le[2] = le[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'lenum Unit not match : ' + le[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[le[1]]] + " : " + " : "+str(le))
                dropobservation[map_itemid_index[le[1]]] +=1
                continue;
        
        #discard none value
        if(le[2] == None):
            writeline(f,'lenum None value :' + str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;
        
        #discard none value
        if(le[2] == None):
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;
        
        wholedata.append(['lenum',(le[0]-admitdate).total_seconds(),list(le)])

    #preprocess chartevents num
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in ('+valid_lab_cate_txt+')')
    labevents = cur.fetchall()
    for le in labevents :    
        
        #tuple to list
        le = list(le)
        
        #check date
        if(le[0] == None):
            writeline(f,'lecate date 0 : ' + " : "+str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;

        #discard none value
        if(le[2] == None):
            writeline(f,'lecate None value :' + str(le))
            dropobservation[map_itemid_index[le[1]]] +=1
            continue;
        
        le[2] = catedict[le[1]][le[2]];
        
        wholedata.append(['lecate',(le[0]-admitdate).total_seconds(),list(le)])
        
        
    #preprocess prescriptions
    cur = conn.cursor()
    cur.execute('SELECT starttime,formulary_drug_cd,dose_val_rx,dose_unit_rx FROM mimiciii.prescriptions WHERE hadm_id = '+str(aid)+' and formulary_drug_cd in ('+valid_prescript_txt+')')
    presevents = cur.fetchall()
    for pe in presevents: 
        #checkdate
        if(pe[0] == None):
            writeline(f,'pe date 0 : '  + " : "+str(pe))
            dropobservation[map_itemid_index[pe[1]]] +=1
            continue;
        
        #tuple to list
        pe = list(pe)
        
        # formatting the value
        dose = pe[2]
        dose = dose.replace(',','').replace(' ','')
        numVal = None;
        try:
            numVal = float(dose)
        except:
            if(len(dose.split('-'))==2):
                strs = dose.split('-')
                try:
                    numVal = (float(strs[0]) + float(strs[1]))/2.0
                except:
                    writeline(f,'pe parse fail : '  + " : "+str(pe))
                    dropobservation[map_itemid_index[pe[1]]] +=1
                    continue;
            else:
                writeline(f,'pe parse fail : '  + " : "+str(pe))
                dropobservation[map_itemid_index[pe[1]]] +=1
                continue;
        
        pe[2] = numVal;
        
        #discard none value
        if(pe[2] == None):
            writeline(f,'PRES None value :' + str(pe))
            dropobservation[map_itemid_index[pe[1]]] +=1
            continue;
            
        #check unit
        currentunit = pe[3].replace(' ','').lower()
        if( currentunit == allitem_unit[map_itemid_index[pe[1]]] or currentunit == ''):
            pass;
        else:    
            gr = findUnitGroup(currentunit)
            if(allitem_unit[map_itemid_index[pe[1]]] in gr):
                targetunit = allitem_unit[map_itemid_index[pe[1]]];
                pe[2] = pe[2] / gr[currentunit] * gr[targetunit]
            else:
                writeline(f,'PRES Unit not match : ' + pe[3].replace(' ','').lower() + ', orig=' + allitem_unit[map_itemid_index[pe[1]]]+ " : "+str(pe))
                continue;

        #discard none value
        if(pe[2] == None):
            writeline(f, 'PRES None value :' + str(pe))
            dropobservation[map_itemid_index[pe[1]]] +=1
            continue;
        
        wholedata.append(['pe',(pe[0]-admitdate).total_seconds(),list(pe)])

    #micro biology preprocess
    for mid in valid_microbio:
        mid = mid.replace('(','').replace(')','')
        m = mid.split(',')
        cur = conn.cursor()
        sql = 'SELECT charttime,(spec_itemid,org_itemid,ab_itemid),dilution_text,\'uom\' FROM mimiciii.microbiologyevents WHERE hadm_id = '+str(aid)
        
        if(m[0] == ''):
            sql += ' and spec_itemid is null'
        else:
            sql += ' and spec_itemid = '+m[0]
    
        if(m[1] == ''):
            sql += ' and org_itemid is null'
        else:
            sql += ' and org_itemid = '+m[1]
        
        if(m[2] == ''):
            sql += ' and ab_itemid is null'
        else:
            sql += ' and ab_itemid = '+m[2]

        cur.execute(sql)
        microevents = cur.fetchall()
        for me in microevents: 
            
            #checkdate
            if(me[0] == None):
                writeline(f,'me date 0 : '+ " : "+str(me))
                dropobservation[map_itemid_index[me[1]]] +=1
                continue;

            #discard none value
            if(me[2] == None):
                writeline(f,'MICRO None value :' + str(me))
                dropobservation[map_itemid_index[me[1]]] +=1
                continue;
            
            #tuple to list
            me = list(me)
            
            #formatting 
            dose = me[2]
            dose = dose.replace('<','').replace('>','').replace('=','')
            numVal = None;
            if(dose == ''):
                writeline(f,'me parse fail : '+ " : "+str(me))
                dropobservation[map_itemid_index[me[1]]] +=1
                continue;
                
            try:
                numVal = float(dose)
            except:
                writeline(f,'me parse fail : '+ " : "+str(me))
                dropobservation[map_itemid_index[me[1]]] +=1
                continue;

            me[2]=numVal;
            
            #discard none value; check again after process
            if(me[2] == None):
                writeline(f,'MICRO None value :' + str(me))
                dropobservation[map_itemid_index[me[1]]] +=1
                continue;
                    
            wholedata.append(['me',(me[0]-admitdate).total_seconds(),list(me)])

    # sort by time
    wholedata = sorted(wholedata, key=itemgetter(1))

    # mapping time to row
    map_time_index = {}
    index = 0;
    for wd in wholedata:
        if(wd[1] not in map_time_index):
            map_time_index[wd[1]] = index
            index += 1;

    # STEP 2 

    # date input output labevent prescript
    D = 2+len(valid_input)+len(valid_output)+len(valid_chart)+len(valid_chart_cate)+len(valid_lab)+len(valid_lab_num)+len(valid_lab_cate)+len(valid_microbio)+len(valid_prescript)

    patient = [[None for i in range(D)] for j in range(len(map_time_index))]
    numtodivide = [[0 for i in range(D-2)] for j in range(len(map_time_index))]
    writeline(f,'len(wholedata) = '+str(len(wholedata)))
    writeline(f, 'D = '+str(D))
    writeline(f,'len(patient) = '+str(len(patient)) +' timesteps')

    for wd in wholedata:

        assert patient[ map_time_index[wd[1]] ][D-2] == None or patient[ map_time_index[wd[1]] ][D-2] == wd[1]
        patient[ map_time_index[wd[1]] ][D-2] = wd[1]
        patient[ map_time_index[wd[1]] ][D-1] = aid

        if(wd[0] ==  'ie' or wd[0] ==  'oe' or wd[0] ==  'pe'):
            if(patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] ==None):
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] = wd[2][2] 
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] += wd[2][2]

        if(wd[0] ==  'le' or wd[0] == 'ce' or wd[0] == 'me' or wd[0] ==  'lenum'):
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]]  ] = wd[2][2]
                numtodivide[map_time_index[wd[1]]  ]  [  map_itemid_index[wd[2][1]] ] = 1
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]]] += wd[2][2]
                numtodivide[ map_time_index[wd[1]]  ] [   map_itemid_index[wd[2][1]] ] += 1
        
        if(wd[0] == 'cecate' or wd[0] == 'lecate'):
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]]  ] = wd[2][2]
            else: 
                print 'DUPLICATED :',wd
                dropobservation[map_itemid_index[wd[2][1]]] +=1
        
    for i in range(len(map_time_index)):
        for j in range(D-2):
            if(numtodivide[i][j] == 0): continue;
            patient[i][j] /= numtodivide[i][j]
    
    return (patient,dropobservation)


In [ ]:
if not os.path.exists(path):
    os.makedirs(path)
if not os.path.exists(path+'/log'):
    os.makedirs(path+'/log')
if not os.path.exists(path+'/dropped'):
    os.makedirs(path+'/dropped')
    
f = open(path+'/log/block-'+str(block),'w')    

cmds = admission_ids[block*1000:(block+1)*1000]
for i in range(len(cmds)):
    aid = cmds[i]
    j = block*1000 + i
    print str(j) + " : " + str(aid)

    res1 = processing(aid,f)
    np.save(path+'/dropped/adm-'+str("%.6d" % j),{'dropped':res1[1],'id':aid,'i':j})
    writeline(f, '--------------------------------------------------------------------------------')
    f.flush()

print 'DONE'

0 : 185131
1 : 164444
2 : 191210
3 : 197882
DUPLICATED : ['cecate', 46200.0, [datetime.datetime(2129, 9, 19, 9, 45), 223758, 2, '']]
DUPLICATED : ['cecate', 46200.0, [datetime.datetime(2129, 9, 19, 9, 45), 223758, 2, '']]
DUPLICATED : ['cecate', 247800.0, [datetime.datetime(2129, 9, 21, 17, 45), 223758, 2, '']]
DUPLICATED : ['cecate', 247800.0, [datetime.datetime(2129, 9, 21, 17, 45), 223758, 2, '']]
DUPLICATED : ['cecate', 305580.0, [datetime.datetime(2129, 9, 22, 9, 48), 223758, 2, '']]
DUPLICATED : ['cecate', 305580.0, [datetime.datetime(2129, 9, 22, 9, 48), 223758, 2, '']]
4 : 140212
DUPLICATED : ['cecate', 34980.0, [datetime.datetime(2180, 4, 19, 7, 12), 223758, 2, '']]
DUPLICATED : ['cecate', 34980.0, [datetime.datetime(2180, 4, 19, 7, 12), 223758, 2, '']]
DUPLICATED : ['cecate', 35160.0, [datetime.datetime(2180, 4, 19, 7, 15), 223758, 2, '']]
DUPLICATED : ['cecate', 35160.0, [datetime.datetime(2180, 4, 19, 7, 15), 223758, 2, '']]
DUPLICATED : ['cecate', 66180.0, [datetime.dateti